In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-11-12 22:07
@Project  :   Hands-on Crawler with Python-decorator
装饰器
'''


# 导入所需的库
import time
from functools import wraps

In [2]:
# 1.简单装饰器
def deco(func):
    def inner():
        print('running inner()')
        func()
    return inner

@deco
def target():
    print('running target()')
    
target() # 语法糖

running inner()
running target()


In [3]:
def target2():
    print('running target2()')

deco(target2)() # 达到相同效果，装饰器可以简化代码

running inner()
running target2()


In [4]:
# 2.装饰器在加载模块时就立即执行
funcs  = []

def add_func(func):
    print(f'add func {func.__name__}')
    funcs.append(func)
    return func

@add_func
def func1():
    print('running func1()')
    
@add_func
def func2():
    print('running func2()')

def func3():
    print('running func3()')

print('running main()') # 装饰器在加载模块时就立即执行，因此在其他函数执行之前运行了2次
print(f'funcs: {funcs}') # func3未被装饰器add_func装饰，因此未添加到列表funcs中
func1() # func1和func2方法只有被明确调用时才执行
func2()
func3()

add func func1
add func func2
running main()
funcs: [<function func1 at 0x0000027822C77F70>, <function func2 at 0x0000027822CA80D0>]
running func1()
running func2()
running func3()


In [5]:
# 3.装饰器实现对程序计时：不改变原函数的前提下为其增加功能
def runtime(func):
    '''计算函数运行时间'''

    def wrapper(*args, **kwargs):  # warpper函数是闭包，使用了自由变量func
        start = time.perf_counter()  # 计时，返回性能计数器的值，包括在睡眠期间和系统范围内流逝的时间，比time.time()精度更高
        res = func(*args, **kwargs)
        time_cost = time.perf_counter() - start
        print(f'function {func.__name__} runtime: {time_cost:.6f} seconds')
        return res

    return wrapper


@runtime
def snooze(seconds=3):
    time.sleep(seconds)


snooze()

function snooze runtime: 3.003628 seconds


In [6]:
snooze(7)

function snooze runtime: 7.002430 seconds


In [7]:
def snooze(seconds=3):
    time.sleep(seconds)


snooze = runtime(snooze)  # 装饰器原理：将wrapper赋值给snooze函数，调用snooze函数其实就是调用wrapper函数
snooze()  # 与前面效果相同

function snooze runtime: 3.005772 seconds


In [8]:
@runtime
def snooze(seconds=3):
    time.sleep(seconds)


snooze.__name__  # 被装饰的函数名称为wrapper，而非原函数的名称。装饰器其实就是把被装饰的函数替换成新的函数并返回被装饰函数本该返回的值，二者接收相同的参数，但新函数通常会做一些额外的操作，例如对函数运行计时等

'wrapper'

In [9]:
# 4.使用functools.wraps装饰器，让装饰器保留原函数的名称
def runtime(func):
    '''计算函数运行时间'''

    @wraps(func)  # wraps装饰器会把被装饰函数的相关属性（__name__、__doc__等）复制到wrapper函数中，因此被装饰函数的名称、文档字符串等会被保留
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        time_cost = time.perf_counter() - start
        print(f'function {func.__name__} runtime: {time_cost:.6f} seconds')
        return res

    return wrapper


@runtime
def snooze(seconds=3):
    time.sleep(seconds)


snooze.__name__

'snooze'

In [10]:
# 5.参数化装饰器：装饰器将悲壮是函数作为第1个参数传入装饰器函数，如果想要接收其他参数则需要构建装饰器工厂函数，将参数传给工厂函数，返回装饰器，再讲装饰器应用到被装饰函数
def runtime(level='info'):  # 工厂函数，返回真正的装饰器
    '''计算函数运行时间'''

    def decorate(func):
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            res = func(*args, **kwargs)
            time_cost = time.perf_counter() - start
            print(f'[{level.upper()}]: function {func.__name__} runtime: {time_cost:.6f} seconds')
            return res

        return wrapper

    return decorate


@runtime(level='debug')  # 必须作为函数调用，才能获得装饰器decorate，等价于runtime(level='debug')(snooze)
def snooze(seconds=3):
    time.sleep(seconds)


snooze()

[DEBUG]: function snooze runtime: 3.010012 seconds


In [11]:
def snooze(seconds=3):
    time.sleep(seconds)


runtime(level='debug')(snooze)()  # 实现相同效果

[DEBUG]: function snooze runtime: 3.000777 seconds


In [12]:
# 6.通过递归实现阶乘并通过装饰器打印出每一次递归调用所需的运行时长
def time_fact(func):
    def wrapper(*args, **kwargs):
        n = args[0]
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'fact({n:2d}) cost time {end - start:.8f} seconds.')
        return res

    return wrapper


@time_fact
def fact(n):
    if n == 1:
        return 1
    else:
        return n * fact(n - 1)


fact(10)

fact( 1) cost time 0.00000060 seconds.
fact( 2) cost time 0.00063480 seconds.
fact( 3) cost time 0.00065140 seconds.
fact( 4) cost time 0.00066010 seconds.
fact( 5) cost time 0.00066820 seconds.
fact( 6) cost time 0.00067600 seconds.
fact( 7) cost time 0.00068350 seconds.
fact( 8) cost time 0.00069090 seconds.
fact( 9) cost time 0.00069820 seconds.
fact(10) cost time 0.00070740 seconds.


3628800